#Import path and library

In [1]:
import pandas as pd
import json
import os
# import xlsxwriter

In [17]:
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p1 = f'{p}Phong /TRANSLATE/'

data_BS = pd.read_excel(f'{p1}BalanceSheet.xlsx', sheet_name='VietStock')
data_IS = pd.read_excel(f'{p1}IncomeStatement.xlsx', sheet_name='VietStock').rename(columns = {'Vietstock_Raw':'VIS_Raw'})
data_CF_Direct = pd.read_excel(f'{p1}CashFlowDirect.xlsx', sheet_name='VietStock')
data_CF_InDirect = pd.read_excel(f'{p1}CashFlowInDirect.xlsx', sheet_name='VietStock')

path_lake = '/content/drive/MyDrive/Data Lake/'
dict_path = json.load(open(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /path.json'))
all_com_1 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_First (1_4).xlsx')['Symbol'])
all_com_2 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase2.xlsx')['Symbol'])
all_com_3 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase3.xlsx')['Symbol'])
all_com_4 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase4.xlsx')['Symbol'])
all_com = all_com_1+all_com_2+all_com_3+all_com_4

In [3]:
data_CF_InDirect['Vietstock_Raw_InDirect'] = data_CF_InDirect['Vietstock_Raw_InDirect'].str.upper()
data_CF_Direct['Vietstock_Raw_Direct'] = data_CF_Direct['Vietstock_Raw_Direct'].str.upper()
data_CF_Direct['Vietstock_Raw_Direct'].iloc[19] = 'LƯU CHUYỂN TIỀN THUẦN TỪ HOẠT ĐỘNG ĐẦU TƯ'
data_CF_InDirect = data_CF_InDirect.rename(columns = {'Vietstock_Raw_InDirect':'LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'})
data_CF_Direct = data_CF_Direct.rename(columns = {'Vietstock_Raw_Direct':'LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'})

#Đổi trường incomstatement

In [6]:
def change_feature_is(symbol, path):
    BS_year = pd.read_csv(f'{path}{symbol}.csv')
    if len(BS_year.index) == 0:
        return BS_year
    BS_year =BS_year[6:].reset_index(drop = True)
    first_col = BS_year.columns[0]
    BS_year  =BS_year.rename(columns = {first_col:'VIS_Raw'})
    # df_concat = pd.concat([data_IS, BS_year], axis = 1)
    df_concat = pd.merge(BS_year, data_BS, how = 'right', on = ['VIS_Raw'])
    if all(df_concat['VIS_Raw'] == df_concat['VIS_Raw']) == True:
        df_BS_year = pd.concat([data_IS, BS_year], axis = 1).drop(columns = ['VIS_Raw', 'Unnamed: 1', 'Unnamed: 2'])
        return df_BS_year
    else:
        return False
# time = 'Year'
# symbol = 'DTL'
# path_in = dict_path['Vietstock_F1'][time]['IS'] + f'{symbol}.csv'
# if os.path.exists(path_in):
#     path_out = dict_path['Vietstock_F2'][time]['IS'] + f'{symbol}.csv'
#     df  =change_feature_is(symbol, dict_path['Vietstock_F1'][time]['IS'])
# df

In [7]:
for i in range(len(all_com)):
    symbol = all_com[i]
    # print(i, symbol, end= ' ')
    for time in ['Quarter', 'Year']:
        path_in = dict_path['Vietstock_F1'][time]['IS'] + f'{symbol}.csv'
        if os.path.exists(path_in):
            path_out = dict_path['Vietstock_F2'][time]['IS'] + f'{symbol}.csv'
            change_feature_is(symbol, dict_path['Vietstock_F1'][time]['IS']).to_csv(path_out, index = False)
        else: continue
    # print('Done')

#Đổi trường BalanceSheet

In [ ]:
def change_feature_bs(symbol, path):
    BS_year = pd.read_csv(f'{path}{symbol}.csv')
    # print(BS_year)
    if len(BS_year.index) == 0:
        return BS_year
    BS_year =BS_year[6:].reset_index(drop = True)
    first_col = BS_year.columns[0]
    BS_year = BS_year.rename(columns = {first_col:'VIS_Raw'})
    # df_concat = pd.concat([data_BS, BS_year], axis = 1)
    df_concat = pd.merge(BS_year, data_BS, how = 'right', on = ['VIS_Raw'])
    # return df_concat
    if all(df_concat['VIS_Raw'] == df_concat['VIS_Raw']) == True:
        df_BS_year = pd.concat([data_BS, BS_year], axis = 1).drop(columns = ['VIS_Raw', 'Unnamed: 1', 'Unnamed: 2', 'Field_Raw'])
        return df_BS_year
    else: 
        return False
time = 'Year'
symbol = 'CCP'
path_in = dict_path['Vietstock_F1'][time]['BS'] + f'{symbol}.csv'
# print(path_in)
if os.path.exists(path_in):
    print('co file')
    change_feature_bs(symbol, dict_path['Vietstock_F1'][time]['BS'])

co file


In [ ]:
for i in range(len(all_com)):
    symbol = all_com[i]
    print(i, symbol, end= ' ')
    for time in ['Quarter', 'Year']:
        print(time)
        path_in = dict_path['Vietstock_F1'][time]['BS'] + f'{symbol}.csv'
        # print(path_in)
        if os.path.exists(path_in):
            path_out = dict_path['Vietstock_F2'][time]['BS'] + f'{symbol}.csv'
            change_feature_bs(symbol, dict_path['Vietstock_F1'][time]['BS']).to_csv(path_out, index = False)
            print(path_out)
        else: continue

In [ ]:
# for symbol in ['ACS', 'BAL', 'BHK', 'BCV', 'BIG', 'BM9', 'BSD', 'BT1', 'CEG', 'DCI', 'DKH', 'DSV']:
#     path_in = dict_path['Vietstock_F1']['Year']['BS']
#     path_out = dict_path['Vietstock_F2']['Year']['BS'] + f'{symbol}.csv'
#     change_feature_bs(symbol, dict_path['Vietstock_F1']['Year']['BS']).to_csv(path_out, index = False)


#Đổi trường Cashflow

In [21]:
def check_symbol(symbol, path):
    df = pd.read_csv(f'{path}{symbol}.csv')
    print(df)
    if len(df.index) == 0:
        return df
    df1 = []
    df2 = []
    check = True
    check1 = []
    check2 = []
    # print(df)
    if len(df) == 96:
        df1 = df[6:56].reset_index(drop = True)
        df2 = df[62:].reset_index(drop = True)
        df1['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'] = df1['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'].str.upper()
        df2['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'] = df2['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'].str.upper()
        df1 = pd.merge(data_CF_InDirect, df1, how = 'left', on = ['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'])
        df2 = pd.merge(data_CF_Direct, df2, how = 'left', on = ['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'])
        df1 = pd.concat([df1, df2]).reset_index(drop = True)
        df1 = df1.drop(columns = ['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP', 'LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP', 'Vietstock_CF_Direct'])
    if len(df) == 56:
        df1 = df[6:].reset_index(drop = True)
        df1['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'] = df1['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'].str.upper()
        df1 = pd.merge(data_CF_InDirect, df1, how = 'left', on = ['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'])
        df1 = df1.drop(columns = ['LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'])
    if len(df) == 40:
        df1 = df[6:].reset_index(drop = True)
        df1['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'] = df1['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'].str.upper()
        df1 = pd.merge(data_CF_Direct, df1, how = 'left', on = ['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'])
        df1 = df1.drop(columns = ['LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP', 'Vietstock_CF_Direct'])
    df1 = df1.drop(columns = ['Unnamed: 1', 'Unnamed: 2'])
    return df1
check_symbol('A32', dict_path['Vietstock_F1']['Quarter']['CF'])

                         LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP  Unnamed: 1  Unnamed: 2
0                                           Giai đoạn         NaN         NaN
1                                            Hợp nhất         NaN         NaN
2                                           Kiểm toán         NaN         NaN
3                                   Công ty kiểm toán         NaN         NaN
4                                    Ý kiến kiểm toán         NaN         NaN
5                                                 NaN         NaN         NaN
6          I. Lưu chuyển tiền từ hoạt động kinh doanh         NaN         NaN
7   1. Tiền thu từ bán hàng, cung cấp dịch v...         NaN         NaN
8   2. Tiền chi trả cho người cung cấp hàng h...         NaN         NaN
9              3. Tiền chi trả cho người lao động         NaN         NaN
10                         4. Tiền chi trả lãi vay         NaN         NaN
11    5. Tiền chi nộp thuế thu nhập doanh nghiệp         Na

KeyError: ignored

In [19]:
for i in range(len(all_com)):
    symbol = all_com[i]
    print(i, symbol, end= ' ')
    for time in ['Quarter', 'Year']:
        print(time)
        path_in = dict_path['Vietstock_F1'][time]['CF'] + f'{symbol}.csv'
        if os.path.exists(path_in):
            path_out = dict_path['Vietstock_F2'][time]['CF'] + f'{symbol}.csv'
            check_symbol(symbol, dict_path['Vietstock_F1'][time]['CF']).to_csv(path_out, index = False)
        else: continue
    # print('Done')

0 24H Quarter
Year
1 A32 Quarter


KeyError: ignored

In [ ]:
# for time in ['Year', 'Quarter']:
#     for finan in ['BS', 'IS', 'CF']:
#         path_in = dict_path['Vietstock_F2'][time][finan]
#         print(time, finan, len(os.listdir(path_in)))